<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/2-mathematical-building-blocks/2_neural_network_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Neural Network from scratch

You should now have a general understanding
of what’s going on behind the scenes in a neural network. What was a magical
black box at the start, has turned into a clearer picture.

<img src='https://github.com/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/2-mathematical-building-blocks/images/2.png?raw=1' width='800'/>

- the model, composed of layers that are chained together, maps the input data to predictions. 
- The loss function then compares these predictions
to the targets, producing a loss value: a measure of how well the model’s predictions match what was expected. 
- The optimizer uses this loss value to update the model’s weights.



##Setup

In [33]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers

import numpy as np
import math

##NN using Keras

Now you understand that the input images are stored in NumPy tensors, which are
here formatted as float32 tensors of shape (60000, 784) (training data) and (10000,
784) (test data) respectively.

In [34]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

Now you understand that this model consists of a chain of two Dense layers, that each
layer applies a few simple tensor operations to the input data, and that these operations
involve weight tensors. Weight tensors, which are attributes of the layers, are
where the knowledge of the model persists.

In [35]:
model = keras.Sequential([
   layers.Dense(512, activation="relu"),
   layers.Dense(10, activation="softmax")                      
])

Now you understand that sparse_categorical_crossentropy is the loss function
that’s used as a feedback signal for learning the weight tensors, and which the training
phase will attempt to minimize. 

You also know that this reduction of the loss
happens via mini-batch stochastic gradient descent. The exact rules governing a specific
use of gradient descent are defined by the rmsprop optimizer passed as the first
argument.

In [36]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Now you understand what happens when you call fit: the model will start to iterate
on the training data in mini-batches of 128 samples, 5 times over (each iteration over
all the training data is called an epoch).

For each batch, the model will compute the
gradient of the loss with regard to the weights (using the Backpropagation algorithm,
which derives from the chain rule in calculus) and move the weights in the direction
that will reduce the value of the loss for this batch.

In [37]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 5s 11ms/step - loss: 0.2553 - accuracy: 0.9258
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.1027 - accuracy: 0.9699
Epoch 3/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0671 - accuracy: 0.9801
Epoch 4/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0493 - accuracy: 0.9855
Epoch 5/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0366 - accuracy: 0.9892


After these 5 epochs, the model will have performed 2,345 gradient updates (469
per epoch), and the loss of the model will be sufficiently low that the model will be
capable of classifying handwritten digits with high accuracy.

At this point, you already know most of what there is to know about neural networks.
Let’s prove it by reimplementing a simplified version of that first example
“from scratch” in TensorFlow, step by step.

##NN from scratch in TensorFlow

What better demonstrates full, unambiguous understanding than implementing everything
from scratch? Of course, what “from scratch” means here is relative: we won’t
reimplement basic tensor operations, and we won’t implement backpropagation. But
we’ll go to such a low level that we will barely use any Keras functionality at all.

###A simple Dense class

You’ve learned earlier that the Dense layer implements the following input transformation,
where W and b are model parameters, and activation is an element-wise
function (usually relu, but it would be softmax for the last layer):

```python
output = activation(dot(W, input) + b)
```

Let’s implement a simple Python class, NaiveDense, that creates two TensorFlow
variables, W and b, and exposes a `__call__()` method that applies the preceding
transformation.

In [38]:
class NaiveDense:

  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    # Create a matrix, W, of shape (input_size, output_size), initialized with random values
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    # Create a vector, b, of shape (output_size,), initialized with zeros
    b_shape = (output_size, )
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  # Convenience method for retrieving the layer’s weights
  @property
  def weights(self):
    return [self.W, self.b]

###A simple Sequential class

Now, let’s create a NaiveSequential class to chain these layers. It wraps a list of layers
and exposes a `__call__()` method that simply calls the underlying layers on the
inputs, in order. 

It also features a weights property to easily keep track of the layers’
parameters.

In [39]:
class NaiveSequential:

  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

Using this NaiveDense class and this NaiveSequential class, we can create a mock
Keras model:

In [40]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)                     
])

assert len(model.weights) == 4

###A batch generator

Next, we need a way to iterate over the MNIST data in mini-batches. This is easy:

In [41]:
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)

    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index: self.index + self.batch_size]
    labels = self.labels[self.index: self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

##Running one training step

The most difficult part of the process is the “training step”: updating the weights of
the model after running it on one batch of data. We need to

1. Compute the predictions of the model for the images in the batch.
2. Compute the loss value for these predictions, given the actual labels.
3. Compute the gradient of the loss with regard to the model’s weights.
4. Move the weights by a small amount in the direction opposite to the gradient.

To compute the gradient, we will use the TensorFlow GradientTape object.

In [42]:
def one_training_step(model, images_path, labels_path):
  # Run the “forward pass” (compute the model’s predictions under a GradientTape scope).
  with tf.GradientTape() as tape:
    predictions = model(images_path)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_path, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)
  
  # Compute the gradient of the loss with regard to the weights. The output gradients is a list where each entry corresponds to a weight from the model.weights list.
  gradients = tape.gradient(average_loss, model.weights)
  # Update the weights using the gradients
  update_weights(gradients, model.weights)

As you already know, the purpose of the “weight update” step (represented by the preceding
update_weights function) is to move the weights by “a bit” in a direction that
will reduce the loss on this batch. 

The magnitude of the move is determined by the
“learning rate,” typically a small quantity. The simplest way to implement this
`update_weights` function is to subtract `gradient * learning_rate` from each weight:

In [43]:
learning_rate = 1e-3

def update_weights(gradients, weights):
  for g, w in zip(gradients, weights):
    # assign_sub is the equivalent of -= for TensorFlow variables
    w.assign_sub(g * learning_rate)

In practice, you would almost never implement a weight update step like this by hand.

Instead, you would use an Optimizer instance from Keras, like this:

In [44]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

def update_weights2(gradients, weights):
  optimizers.apply_gradients(zip(gradients, weights))

Now that our per-batch training step is ready, we can move on to implementing an
entire epoch of training.

##The full training loop

An epoch of training simply consists of repeating the training step for each batch in
the training data, and the full training loop is simply the repetition of one epoch:

In [45]:
def fit(model, images, labels, epochs, batch_size=128):
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"loss at batch {batch_counter}: {loss:.2f}")

Let’s test drive it:

In [46]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [47]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0


TypeError: ignored

##Evaluating the model

We can evaluate the model by taking the argmax of its predictions over the test images,
and comparing it to the expected labels:

In [ ]:
predictions = model(test_images)
predictions = predictions.numpy()

predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels

print(f"accuracy: {matches.mean(): .2f}")

All done! As you can see, it’s quite a bit of work to do “by hand” what you can do in a
few lines of Keras code. But because you’ve gone through these steps, you should now
have a crystal clear understanding of what goes on inside a neural network when you
call fit(). 

Having this low-level mental model of what your code is doing behind the
scenes will make you better able to leverage the high-level features of the Keras API.